In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as scp
from scipy import stats
import sklearn
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import re

In [4]:
pd.options.display.max_columns = 36

In [31]:
df_inc = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Census/2017_block_income_ann.csv')
df_pop = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Census/2017_block_pop_with_ann.csv', header = None)
df_land_area = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Census/2010_tract_landarea_ann.csv', header = None)

In [32]:
df_inc.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,HD01_VD06,HD02_VD06,HD01_VD07,HD02_VD07,HD01_VD08,...,HD01_VD09,HD02_VD09,HD01_VD10,HD02_VD10,HD01_VD11,HD02_VD11,HD01_VD12,HD02_VD12,HD01_VD13,HD02_VD13,HD01_VD14,HD02_VD14,HD01_VD15,HD02_VD15,HD01_VD16,HD02_VD16,HD01_VD17,HD02_VD17
0,Id,Id2,Geography,Estimate; Total:,Margin of Error; Total:,"Estimate; Total: - Less than $10,000","Margin of Error; Total: - Less than $10,000","Estimate; Total: - $10,000 to $14,999","Margin of Error; Total: - $10,000 to $14,999","Estimate; Total: - $15,000 to $19,999","Margin of Error; Total: - $15,000 to $19,999","Estimate; Total: - $20,000 to $24,999","Margin of Error; Total: - $20,000 to $24,999","Estimate; Total: - $25,000 to $29,999","Margin of Error; Total: - $25,000 to $29,999","Estimate; Total: - $30,000 to $34,999","Margin of Error; Total: - $30,000 to $34,999","Estimate; Total: - $35,000 to $39,999",...,"Estimate; Total: - $40,000 to $44,999","Margin of Error; Total: - $40,000 to $44,999","Estimate; Total: - $45,000 to $49,999","Margin of Error; Total: - $45,000 to $49,999","Estimate; Total: - $50,000 to $59,999","Margin of Error; Total: - $50,000 to $59,999","Estimate; Total: - $60,000 to $74,999","Margin of Error; Total: - $60,000 to $74,999","Estimate; Total: - $75,000 to $99,999","Margin of Error; Total: - $75,000 to $99,999","Estimate; Total: - $100,000 to $124,999","Margin of Error; Total: - $100,000 to $124,999","Estimate; Total: - $125,000 to $149,999","Margin of Error; Total: - $125,000 to $149,999","Estimate; Total: - $150,000 to $199,999","Margin of Error; Total: - $150,000 to $199,999","Estimate; Total: - $200,000 or more","Margin of Error; Total: - $200,000 or more"
1,1500000US130079601001,130079601001,"Block Group 1, Census Tract 9601, Baker County...",378,77,88,38,47,29,6,10,9,14,18,16,28,28,7,...,14,18,16,17,22,18,40,32,44,36,22,20,10,14,7,11,0,13
2,1500000US130079601002,130079601002,"Block Group 2, Census Tract 9601, Baker County...",426,71,88,59,7,10,7,11,33,25,8,10,7,9,9,...,44,41,14,21,79,46,53,39,12,14,45,36,13,17,3,5,4,6
3,1500000US130079602001,130079602001,"Block Group 1, Census Tract 9602, Baker County...",224,63,0,13,7,10,7,10,27,26,13,18,26,28,0,...,5,8,25,39,36,34,27,21,30,27,0,13,0,13,0,13,21,20
4,1500000US130079602002,130079602002,"Block Group 2, Census Tract 9602, Baker County...",293,64,55,48,0,13,10,11,23,23,44,39,24,22,14,...,0,13,18,33,30,32,5,8,36,25,0,13,0,13,34,30,0,13


In [33]:
df_inc['GEO.display-label']

0                                              Geography
1      Block Group 1, Census Tract 9601, Baker County...
2      Block Group 2, Census Tract 9601, Baker County...
3      Block Group 1, Census Tract 9602, Baker County...
4      Block Group 2, Census Tract 9602, Baker County...
5      Block Group 1, Census Tract 1, Dougherty Count...
6      Block Group 2, Census Tract 1, Dougherty Count...
7      Block Group 3, Census Tract 1, Dougherty Count...
8      Block Group 4, Census Tract 1, Dougherty Count...
9      Block Group 5, Census Tract 1, Dougherty Count...
10     Block Group 1, Census Tract 2, Dougherty Count...
11     Block Group 2, Census Tract 2, Dougherty Count...
12     Block Group 3, Census Tract 2, Dougherty Count...
13     Block Group 4, Census Tract 2, Dougherty Count...
14     Block Group 1, Census Tract 4, Dougherty Count...
15     Block Group 2, Census Tract 4, Dougherty Count...
16     Block Group 3, Census Tract 4, Dougherty Count...
17     Block Group 1, Census Tr

In [38]:
new = df_inc["GEO.display-label"].str.split(", ", expand = True)
df_inc["block"] = new[0] 
df_inc["tract"] = new[1] 
df_inc["county"] = new[2] 

In [40]:
df_inc = df_inc.drop('GEO.display-label', axis = 1)

In [42]:
l = len(df_inc['tract'])
df_inc1 = df_inc.copy()
re_string = 'Census Tract (\d+(?:.\d+)?)'
for i in range(1,l):
    s = df_inc.loc[i, 'tract']
    m = re.search(re_string, s)
    df_inc1.loc[i, 'tract'] = m.group(1)

In [44]:
l = len(df_inc['block'])
re_string = 'Block Group (\d+(?:.\d+)?)'
for i in range(1,l):
    s = df_inc.loc[i, 'block']
    m = re.search(re_string, s)
    df_inc1.loc[i, 'block'] = m.group(1)

In [48]:
tract_series = df_inc1['tract']
l = len(df_inc1['tract'])
for i in range(1,l):
    if '.' in tract_series[i]:
        s = tract_series[i].split('.')
        if len(s[0]) == 1:
            tract_series[i] = s[0] + s[1]
        elif len(s[0]) == 2:
            tract_series[i] = s[0] + s[1]
        elif len(s[0]) == 3:
            tract_series[i] = s[0] + s[1]
    else:
        if len(tract_series[i]) == 3:
            tract_series[i] = df_inc1['tract'][i] + '00'
        elif len(tract_series[i]) == 2:
            tract_series[i] = df_inc1['tract'][i] + '00'
        elif len(tract_series[i]) == 1:
            tract_series[i] = df_inc1['tract'][i] + '00'
        elif len(tract_series[i]) == 4:
            tract_series[i] = df_inc1['tract'][i] + '00'

In [49]:
df_inc1.head(20)

,GEO.id,GEO.id2,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,HD01_VD06,HD02_VD06,HD01_VD07,HD02_VD07,HD01_VD08,HD02_VD08,...,HD02_VD10,HD01_VD11,HD02_VD11,HD01_VD12,HD02_VD12,HD01_VD13,HD02_VD13,HD01_VD14,HD02_VD14,HD01_VD15,HD02_VD15,HD01_VD16,HD02_VD16,HD01_VD17,HD02_VD17,block,tract,county
0,Id,Id2,Estimate; Total:,Margin of Error; Total:,"Estimate; Total: - Less than $10,000","Margin of Error; Total: - Less than $10,000","Estimate; Total: - $10,000 to $14,999","Margin of Error; Total: - $10,000 to $14,999","Estimate; Total: - $15,000 to $19,999","Margin of Error; Total: - $15,000 to $19,999","Estimate; Total: - $20,000 to $24,999","Margin of Error; Total: - $20,000 to $24,999","Estimate; Total: - $25,000 to $29,999","Margin of Error; Total: - $25,000 to $29,999","Estimate; Total: - $30,000 to $34,999","Margin of Error; Total: - $30,000 to $34,999","Estimate; Total: - $35,000 to $39,999","Margin of Error; Total: - $35,000 to $39,999",...,"Margin of Error; Total: - $45,000 to $49,999","Estimate; Total: - $50,000 to $59,999","Margin of Error; Total: - $50,000 to $59,999","Estimate; Total: - $60,000 to $74,999","Margin of Error; Total: - $60,000 to $74,999","Estimate; Total: - $75,000 to $99,999","Margin of Error; Total: - $75,000 to $99,999","Estimate; Total: - $100,000 to $124,999","Margin of Error; Total: - $100,000 to $124,999","Estimate; Total: - $125,000 to $149,999","Margin of Error; Total: - $125,000 to $149,999","Estimate; Total: - $150,000 to $199,999","Margin of Error; Total: - $150,000 to $199,999","Estimate; Total: - $200,000 or more","Margin of Error; Total: - $200,000 or more",Geography,None,None
1,1500000US130079601001,130079601001,378,77,88,38,47,29,6,10,9,14,18,16,28,28,7,13,...,17,22,18,40,32,44,36,22,20,10,14,7,11,0,13,1,960100,Baker County
2,1500000US130079601002,130079601002,426,71,88,59,7,10,7,11,33,25,8,10,7,9,9,10,...,21,79,46,53,39,12,14,45,36,13,17,3,5,4,6,2,960100,Baker County
3,1500000US130079602001,130079602001,224,63,0,13,7,10,7,10,27,26,13,18,26,28,0,13,...,39,36,34,27,21,30,27,0,13,0,13,0,13,21,20,1,960200,Baker County
4,1500000US130079602002,130079602002,293,64,55,48,0,13,10,11,23,23,44,39,24,22,14,15,...,33,30,32,5,8,36,25,0,13,0,13,34,30,0,13,2,960200,Baker County
5,1500000US130950001001,130950001001,280,124,0,13,0,13,0,13,9,18,50,64,13,21,15,23,...,56,26,43,44,66,0,13,0,13,0,13,0,13,0,13,1,100,Dougherty County
6,1500000US130950001002,130950001002,175,98,16,26,32,32,12,19,0,13,65,69,35,56,0,13,...,13,0,13,0,13,0,13,0,13,0,13,0,13,0,13,2,100,Dougherty County
7,1500000US130950001003,130950001003,329,118,121,77,0,13,28,43,62,65,16,25,25,41,27,41,...,13,31,39,0,13,0,13,0,13,0,13,0,13,0,13,3,100,Dougherty County
8,1500000US130950001004,130950001004,714,180,135,99,51,60,103,82,136,112,60,65,0,13,46,59,...,68,63,61,0,13,55,53,12,19,0,13,0,13,0,13,4,100,Dougherty County
9,1500000US130950001005,130950001005,407,140,90,77,37,37,10,16,13,22,17,21,131,122,0,13,...,34,8,18,11,19,13,23,0,13,0,13,44,66,0,13,5,100,Dougherty County
